# Librosa
## This notebook outlines the installation and usage of Librosa library for Audio Processing

### Installation

In [1]:
# ! pip install librosa

     |████████████████████████████████| 203 kB 4.5 MB/s            
     |████████████████████████████████| 613 kB 44.1 MB/s            
     |████████████████████████████████| 323 kB 67.5 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 57 kB 10.9 MB/s            
     |████████████████████████████████| 377 kB 48.3 MB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23143 sha256=f614501f33d66b173ba7ac8f8bdd3c97573dd01c20c94f0aeb979680bd999c90
  Stored in directory: /Users/daniyarkurmanbayev/Library/Caches/pip/wheels/49/5a/e4/df590783499a992a88de6c0898991d1167453a3196d0d1eeb7
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320719 sha256=2d86079df42415c9121d63fc807be1f38b2350ecfc4b4bfb279a034c22959c7b
  Stored in directory: /Users/daniyarkurmanbayev/Library/Caches/pip/wheels/6f/d1/5d/f13da53b1dcbc2624ff548456c9ffb526c914f5

### Import the library

In [2]:
import librosa
import numpy as np

### Get the file path to an included audio example
- Use nutcracker

In [3]:
filename = librosa.example('nutcracker')

### Load the audio 
- Store the audio as a waveform `y`
- Store the sampling rate as `sr`

In [4]:
y, sr = librosa.load(filename)

### Set the hop length; at 22050 Hz, 512 samples ~ 23ms

In [5]:
hop_length = 512

### Separate harmonics and percussives into two waveforms

In [6]:
y_harmonic, y_percussive = librosa.effects.hpss(y)

In [7]:
y_percussive

array([ 4.2039378e-06,  1.7493697e-06, -9.4658744e-06, ...,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00], dtype=float32)

In [8]:
y_harmonic

array([-1.9322506e-06,  3.5833509e-06,  2.2185454e-06, ...,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00], dtype=float32)

### Run the default beat tracker

In [9]:
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

In [10]:
print(f'Estimated tempo: {tempo:.2f} beats per minute')

Estimated tempo: 107.67 beats per minute


### Convert the frame indices of beat events into timestamps

In [11]:
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

In [12]:
beat_times

array([  1.18421769,   1.71827664,   2.32199546,   2.87927438,
         3.45977324,   4.01705215,   4.59755102,   5.13160998,
         5.71210884,   6.26938776,   6.84988662,   7.40716553,
         7.9876644 ,   8.52172336,   9.12544218,   9.63628118,
        10.19356009,  10.70439909,  11.261678  ,  11.77251701,
        12.32979592,  12.86385488,  13.39791383,  13.93197279,
        14.46603175,  15.0000907 ,  15.53414966,  16.06820862,
        16.62548753,  17.15954649,  17.69360544,  18.2276644 ,
        18.7385034 ,  19.29578231,  19.85306122,  20.36390023,
        20.89795918,  21.43201814,  21.9660771 ,  22.50013605,
        23.05741497,  23.56825397,  24.10231293,  24.63637188,
        25.19365079,  25.7044898 ,  26.26176871,  26.81904762,
        27.35310658,  27.88716553,  28.44444444,  28.9785034 ,
        29.53578231,  30.09306122,  30.65034014,  31.20761905,
        31.76489796,  32.32217687,  32.83301587,  33.36707483,
        33.90113379,  34.43519274,  34.94603175,  35.45

### Compute MFCC features from the raw signal

In [13]:
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

### And the first-order differences (delta features)

In [14]:
mfcc_delta = librosa.feature.delta(mfcc)

### Stack and synchronize between beat events
- Use the mean value (default) instead of median

In [15]:
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)

### Compute chroma features from the harmonic signal

In [16]:
chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                        sr=sr)

### Aggregate chroma features between beat events
- Use the median value of each feature between beat frames

In [17]:
beat_chroma = librosa.util.sync(chromagram,
                                beat_frames,
                                aggregate=np.median)

### Stack all beat-synchronous features together


In [18]:
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])